In [1]:
from lxml import html
import requests
import pandas as pd

df = pd.DataFrame()

In [2]:
#scraping NFL 2014-2017 seasons

for year in range(2014,2018):
    url = 'https://www.pro-football-reference.com/years/%d/games.htm' % year
    page = requests.get(url)

# build the DOM Tree
    tree = html.fromstring(page.content)

#get data
    data = tree.xpath('//*[(@id = "games")]//a | //*[(@id = "games")]//strong | //td | //*[contains(concat( " ", @class, " " ), concat( " ", "right", " " ))]')

    row = []
    r=0
    
#dealing with imperfect scraping

    for x in range(len(data)):
        # dealing with ties
        if ((year==2014)&((x==1600)|(x==1606))):
            row.append('None')
            r = 17
        if ((year==2016)&((x==1999)|(x==2005))):
            row.append('None')
            r = 17
        if ((year==2016)&((x==2054)|(x==2060))):
            row.append('None')
            r = 15
        if (x%19==r) & (x>0):
            row.append(year)
            df = df.append([row])
            row = []
        row.append(data[x].text)

    df = df.iloc[:-11]

In [3]:
# deleting extra/ unnecessary columns from scape

df = df.drop([1,2,3,4,5,8,10,11,12], axis=1)

In [4]:
# more readable column names
df.columns = ['Week', 'Win_team', 'Home_Away','Lose_team', 'Win_pts', 'Lose_pts','Win_yds','Win_TO','Lose_yds','Lose_TO','Season']


In [5]:
def home(x):
    wteam = x[0]
    ha = x[1]
    lteam = x[2]
    if (ha=='@'):
        ht = lteam
    else:
        ht = wteam
    return ht

def away(x):
    wteam = x[0]
    ha = x[1]
    lteam = x[2]
    if (ha=='@'):
        at = wteam
    else:
        at = lteam
    return at

df['Home_team'] = df[['Win_team','Home_Away','Lose_team']].apply(home, axis=1)
df['Away_team'] = df[['Win_team','Home_Away','Lose_team']].apply(away, axis=1)

In [6]:
# functions for converting data from winning or losing team to home or away team

def home_cols(x):
    wt = x[0]
    lt = x[1]
    ht = x[2]
    at = x[3]
    wcol = x[4]
    lcol = x[5]
    if wt==ht:
        home = wcol
    elif lt==ht:
        home = lcol
    return home

def away_cols(x):
    wt = x[0]
    lt = x[1]
    ht = x[2]
    at = x[3]
    wcol = x[4]
    lcol = x[5]
    if wt==at:
        away = wcol
    elif lt==at:
        away = lcol
    return away

df['Home_pts_for'] = df[['Win_team','Lose_team','Home_team','Away_team', 'Win_pts','Lose_pts']].apply(home_cols, axis=1)
df['Home_pts_against'] = df[['Win_team','Lose_team','Home_team','Away_team', 'Win_pts','Lose_pts']].apply(away_cols, axis=1)
df['Away_pts_for'] = df[['Win_team','Lose_team','Home_team','Away_team', 'Win_pts','Lose_pts']].apply(away_cols, axis=1)
df['Away_pts_against'] = df[['Win_team','Lose_team','Home_team','Away_team', 'Win_pts','Lose_pts']].apply(home_cols, axis=1)

df['Home_yds_for'] = df[['Win_team','Lose_team','Home_team','Away_team','Win_yds','Lose_yds']].apply(home_cols, axis=1)
df['Home_yds_against'] = df[['Win_team','Lose_team','Home_team','Away_team', 'Win_yds','Lose_yds']].apply(away_cols, axis=1)
df['Away_yds_for'] = df[['Win_team','Lose_team','Home_team','Away_team', 'Win_yds','Lose_yds']].apply(away_cols, axis=1)
df['Away_yds_against'] = df[['Win_team','Lose_team','Home_team','Away_team','Win_yds','Lose_yds']].apply(home_cols, axis=1)

df['Home_TO_for'] = df[['Win_team','Lose_team','Home_team','Away_team','Win_TO','Lose_TO']].apply(home_cols, axis=1)
df['Home_TO_against'] = df[['Win_team','Lose_team','Home_team','Away_team', 'Win_TO','Lose_TO']].apply(away_cols, axis=1)
df['Away_TO_for'] = df[['Win_team','Lose_team','Home_team','Away_team', 'Win_TO','Lose_TO']].apply(away_cols, axis=1)
df['Away_TO_against'] = df[['Win_team','Lose_team','Home_team','Away_team','Win_TO','Lose_TO']].apply(home_cols, axis=1)

In [7]:
# change data type so it can be manipulated

df[['Home_pts_for', 'Home_pts_against',
       'Away_pts_for', 'Away_pts_against', 'Home_yds_for', 'Home_yds_against',
       'Away_yds_for', 'Away_yds_against', 'Home_TO_for', 'Home_TO_against',
       'Away_TO_for', 'Away_TO_against']] = df[['Home_pts_for', 'Home_pts_against',
       'Away_pts_for', 'Away_pts_against', 'Home_yds_for', 'Home_yds_against',
       'Away_yds_for', 'Away_yds_against', 'Home_TO_for', 'Home_TO_against',
       'Away_TO_for', 'Away_TO_against']].astype(float)

df[['Week','Season']]=df[['Week','Season']].astype(int)

In [8]:
# columns for wins and losses

def wins(x):
    team = x[0]
    week = x[1]
    season = x[2]
    df_t = df[(df['Win_team']==team)&(df['Season']==season)&(df['Week']<week)]
    w = len(df_t)
    return w

def loss(x):
    team = x[0]
    week = x[1]
    season = x[2]
    df_t = df[(df['Lose_team']==team)&(df['Season']==season)&(df['Week']<week)]
    w = len(df_t)
    return w

df['Home_wins'] = df[['Home_team','Week','Season']].apply(wins, axis=1)
df['Home_losses'] = df[['Home_team','Week','Season']].apply(loss, axis=1)
df['Away_wins'] = df[['Away_team','Week','Season']].apply(wins, axis=1)
df['Away_losses'] = df[['Away_team','Week','Season']].apply(loss, axis=1)

In [9]:
def home_win(x):
    wt=x[0]
    ht=x[1]
    win = 0
    if wt==ht:
        win=1
    else:
        win=0
    return win

df['Home_win']=df[['Win_team','Home_team']].apply(home_win,axis=1)

In [10]:
#drop win lose columns

df = df.drop(['Win_team','Home_Away','Lose_team','Win_pts','Lose_pts','Win_yds','Win_TO','Lose_yds','Lose_TO'], axis=1)

In [11]:
# functions for rolling averages 

def r_a_ppg_for(x):
    team = x[0]
    week = x[1]
    season = x[2]
    
    df_t = df[((df['Home_team']==team)|(df['Away_team']==team))&(df['Season']==season)&(df['Week']<week)]
    df_t.reset_index(drop=True,inplace=True)
    r_a = 0
    for row in range(len(df_t)):
        if df_t['Home_team'][row]==team:
            r_a+=df_t['Home_pts_for'][row]
        elif df_t['Away_team'][row]==team:
            r_a+=df_t['Away_pts_for'][row]
    try:
        r_a = r_a/len(df_t)
    except:
        r_a = 0
    return r_a

def r_a_ppg_against(x):
    team = x[0]
    week = x[1]
    season = x[2]
    
    df_t = df[((df['Home_team']==team)|(df['Away_team']==team))&(df['Season']==season)&(df['Week']<week)]
    df_t.reset_index(drop=True,inplace=True)
    r_a = 0
    for row in range(len(df_t)):
        if df_t['Home_team'][row]==team:
            r_a+=df_t['Away_pts_for'][row]
        elif df_t['Away_team'][row]==team:
            r_a+=df_t['Home_pts_for'][row]
    try:
        r_a = r_a/len(df_t)
    except:
        r_a = 0
    return r_a

df['Home_ppg_for'] = df[['Home_team','Week','Season']].apply(r_a_ppg_for, axis=1)
df['Away_ppg_for'] = df[['Away_team','Week','Season']].apply(r_a_ppg_for, axis=1)
df['Home_ppg_against'] = df[['Home_team','Week','Season']].apply(r_a_ppg_against, axis=1)
df['Away_ppg_against'] = df[['Away_team','Week','Season']].apply(r_a_ppg_against, axis=1)



def r_a_ypg_for(x):
    team = x[0]
    week = x[1]
    season = x[2]
    
    df_t = df[((df['Home_team']==team)|(df['Away_team']==team))&(df['Season']==season)&(df['Week']<week)]
    df_t.reset_index(drop=True,inplace=True)
    r_a = 0
    for row in range(len(df_t)):
        if df_t['Home_team'][row]==team:
            r_a+=df_t['Home_yds_for'][row]
        elif df_t['Away_team'][row]==team:
            r_a+=df_t['Away_yds_for'][row]
    try:
        r_a = r_a/len(df_t)
    except:
        r_a = 0
    return r_a

def r_a_ypg_against(x):
    team = x[0]
    week = x[1]
    season = x[2]
    
    df_t = df[((df['Home_team']==team)|(df['Away_team']==team))&(df['Season']==season)&(df['Week']<week)]
    df_t.reset_index(drop=True,inplace=True)
    r_a = 0
    for row in range(len(df_t)):
        if df_t['Home_team'][row]==team:
            r_a+=df_t['Away_yds_for'][row]
        elif df_t['Away_team'][row]==team:
            r_a+=df_t['Home_yds_for'][row]
    try:
        r_a = r_a/len(df_t)
    except:
        r_a = 0
    return r_a

df['Home_ypg_for'] = df[['Home_team','Week','Season']].apply(r_a_ypg_for, axis=1)
df['Away_ypg_for'] = df[['Away_team','Week','Season']].apply(r_a_ypg_for, axis=1)
df['Home_ypg_against'] = df[['Home_team','Week','Season']].apply(r_a_ypg_against, axis=1)
df['Away_ypg_against'] = df[['Away_team','Week','Season']].apply(r_a_ypg_against, axis=1)



def to_for(x):
    team = x[0]
    week = x[1]
    season = x[2]
    
    df_t = df[((df['Home_team']==team)|(df['Away_team']==team))&(df['Season']==season)&(df['Week']<week)]
    df_t.reset_index(drop=True,inplace=True)
    to = 0
    for row in range(len(df_t)):
        if df_t['Home_team'][row]==team:
            to+=df_t['Home_TO_for'][row]
        elif df_t['Away_team'][row]==team:
            to+=df_t['Away_TO_for'][row]
    return to

def to_against(x):
    team = x[0]
    week = x[1]
    season = x[2]
    
    df_t = df[((df['Home_team']==team)|(df['Away_team']==team))&(df['Season']==season)&(df['Week']<week)]
    df_t.reset_index(drop=True,inplace=True)
    to = 0
    for row in range(len(df_t)):
        if df_t['Home_team'][row]==team:
            to+=df_t['Away_TO_for'][row]
        elif df_t['Away_team'][row]==team:
            to+=df_t['Home_TO_for'][row]
    return to

df['Home_TO_for_total'] = df[['Home_team','Week','Season']].apply(to_for, axis=1)
df['Away_TO_for_total'] = df[['Away_team','Week','Season']].apply(to_for, axis=1)
df['Home_TO_against_total'] = df[['Home_team','Week','Season']].apply(to_against, axis=1)
df['Away_TO_against_total'] = df[['Away_team','Week','Season']].apply(to_against, axis=1)

In [12]:
#drop non rolling average columns

df = df.drop(['Home_pts_for',
       'Home_pts_against', 'Away_pts_for', 'Away_pts_against', 'Home_yds_for',
       'Home_yds_against', 'Away_yds_for', 'Away_yds_against', 'Home_TO_for',
       'Home_TO_against', 'Away_TO_for', 'Away_TO_against'], axis=1)

In [13]:
# mapping full names to abbreviations so they can match with other dataset
    
abr = {'Seattle Seahawks':'SEA', 'Chicago Bears':'CHI', 'St. Louis Rams':'LAR',
       'Kansas City Chiefs':'KAN', 'Baltimore Ravens':'BAL', 'Philadelphia Eagles':'PHI',
       'Pittsburgh Steelers':'PIT', 'New York Jets':'NYJ', 'Atlanta Falcons':'ATL',
       'Miami Dolphins':'MIA', 'Houston Texans':'HOU', 'Tampa Bay Buccaneers':'TAM',
       'Dallas Cowboys':'DAL', 'Denver Broncos':'DEN', 'Detroit Lions':'DET',
       'Arizona Cardinals':'ARI', 'Buffalo Bills':'BUF', 'Cleveland Browns':'CLE',
       'Tennessee Titans':'TEN', 'New York Giants':'NYG', 'Carolina Panthers':'CAR',
       'Washington Redskins':'WAS', 'Minnesota Vikings':'MIN', 'Cincinnati Bengals':'CIN',
       'San Diego Chargers':'LAC', 'Oakland Raiders':'OAK', 'Green Bay Packers':'GNB',
       'San Francisco 49ers':'SFO', 'Indianapolis Colts':'IND', 'Jacksonville Jaguars':'JAX',
       'New England Patriots':'NWE', 'New Orleans Saints':'NOR', 'Los Angeles Rams':'LAR',
       'Los Angeles Chargers':'LAC'}

df['Home_team'] = df['Home_team'].map(abr)
df['Away_team'] = df['Away_team'].map(abr)

In [14]:
#turnover difference columns

df['Home_TO_diff'] = df['Home_TO_for_total']-df['Home_TO_against_total']
df['Away_TO_diff'] = df['Away_TO_for_total']-df['Away_TO_against_total']

In [15]:
# csv file so easily merged with other dataframe

df.to_csv('NFL_scrape.csv')